In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'SDC_EDC_2.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


dfs = pd.read_csv('datasets/SDC_EDC/SDC.txt', header=None, names=['text'], encoding='utf-8', engine='python')
dfe = pd.read_csv('datasets/SDC_EDC/EDC.txt', header=None, names=['text'], encoding='utf-8', engine='python')

dfs['label'] = 0
dfe['label'] = 1


df = pd.concat([dfs, dfe], ignore_index=True)
display(len(df))
 
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

display(len(df))
display(df.columns)
display(df[:4])

classes_num = 2
display(classes_num)

ds = Dataset.from_pandas(df)

ds = ds.train_test_split(test_size=0.2)
display(ds)

max_sequence_length = 128


models = [ 
        'faisalq/EgyBERT',
        'UBC-NLP/MARBERT',
        'UBC-NLP/MARBERTv2',  
]


for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds['train']
        dataset_validation = ds['test']                                                    
        
      

        def preprocess_function(examples):
            return tokenizer(examples['text'], truncation=True, padding="max_length",
                            max_length=max_sequence_length)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 5
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 64
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 50, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 50
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('SDC_EDC_results_2.csv')
display(best_results)



2024-07-25 03:20:36.128218: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 03:20:36.151230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 03:20:36.503227: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


28613

28613

Index(['text', 'label'], dtype='object')

,text,label
0,لما تـسيب الفلاني و المسلسل الفلاني والأغاني اللي بتحبها عشان ربنا وبس,1
1,وانت محل ما اروح ورايااا انا ايش قلت عنك دحين,0
2,فاذا حد مرت عليه نفس السالفة يفيدني بالله اديني خبر اول ما تدخل لأَنِّي انا عندي نفس المشكله ولي سمعتوا عادي لو باقي يوم حتا مين جرب تعرفون احد جاي الله يعافيكم الي يعرف ايش الحل مع العلم انو جاني لاكن يوم ارجع اشيك انا كمان اسأل رجع تاني ع يا جماعة الخير إنّو ما فيها اول شي انصحك تبعدين عن,0
3,وفي مجلس الحريم بعد ماطلعت كلمت وقالتلها على الموضوع فرحت ورحبت بالموضوع لانه واهله ناس طيبين ومعروفين بأخلاقهم واصلهم وقالتلها انها تستشير وترد لهم خبر,0


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/22890 [00:00<?, ? examples/s]

Map:   0%|          | 0/5723 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
50,0.272700,0.137633,0.946532,0.946531
100,0.163600,0.138186,0.962956,0.962944
150,0.125900,0.100379,0.967150,0.967130
200,0.099300,0.101239,0.971344,0.971331
250,0.100500,0.145083,0.963306,0.963153
300,0.092500,0.074018,0.972043,0.971999
350,0.093300,0.075135,0.975363,0.975308
400,0.053900,0.183698,0.963306,0.963304
450,0.063600,0.090462,0.975537,0.975501
500,0.063900,0.073949,0.977285,0.977239


,Model,Accuracy,F1
0,UBC-NLP/MARBERT,0.974139,0.974104
3,UBC-NLP/MARBERTv2,0.978857,0.978825
6,faisalq/EgyBERT,0.980954,0.980932
